In [2]:
!python3 --version

Python 3.8.6


In [3]:
import pandas as pd
import seaborn as sns

In [5]:
print('ready for some more eth data?')

ready for some more eth data?


In [19]:
data = 'data/09_16_20/attestations_1.csv' 

In [20]:
df = pd.read_csv(data)

In [23]:
df.head()

,slot,beacon_block_root,attesting_indices,source_epoch,source_block_root,target_epoch,target_block_root
0,64590,0x2622bb6825cbd8041bfb719ecc8cec31a8e39881a472...,0000000000000000000000000001000000001000000000...,2017,0xd0fe548ffd8197f58814300ed02471ae8d386e0fc24e...,2018,0xabcbd38d70fd0123c136a4593d820bdef1fc94251a44...
1,64590,0x3de5c03066e9c33694f5af05d8c2150fc634ed96bffe...,1011100101101011001111010110111101001111111111...,2017,0xd0fe548ffd8197f58814300ed02471ae8d386e0fc24e...,2018,0xabcbd38d70fd0123c136a4593d820bdef1fc94251a44...
2,64590,0x3de5c03066e9c33694f5af05d8c2150fc634ed96bffe...,0011110100111110101011011110100111001011000111...,2017,0xd0fe548ffd8197f58814300ed02471ae8d386e0fc24e...,2018,0xabcbd38d70fd0123c136a4593d820bdef1fc94251a44...
3,64590,0x2622bb6825cbd8041bfb719ecc8cec31a8e39881a472...,0000000000000000000000000000000000000000000000...,2017,0xd0fe548ffd8197f58814300ed02471ae8d386e0fc24e...,2018,0xabcbd38d70fd0123c136a4593d820bdef1fc94251a44...
4,64590,0x2622bb6825cbd8041bfb719ecc8cec31a8e39881a472...,0000000000000000000000000000001000000000000000...,2017,0xd0fe548ffd8197f58814300ed02471ae8d386e0fc24e...,2018,0xabcbd38d70fd0123c136a4593d820bdef1fc94251a44...
